In [1]:
import tensorflow as tf

2023-10-06 02:11:45.879469: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 02:11:45.987173: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 02:11:45.989902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 02:11:47.557018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
example_path="gs://yejingxin-us-central2/external/lg/dummy-data/train/*.tfrecords"

In [12]:
one_data_path="gs://yejingxin-us-central2/external/lg/dummy-data/train/LG_GPT_KO2_8_11_100000.tfrecords"

In [13]:
# dataset = tf.data.TFRecordDataset.list_files(example_path)  
dataset = tf.data.TFRecordDataset(one_data_path)


In [14]:
dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [23]:
for raw_record in dataset.take(1):
    print(raw_record.numpy().decode("utf-8"))
    # print(tf.io.decode_raw(raw_record, None))


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 1: invalid continuation byte

In [33]:
# Define a feature description for parsing the serialized examples  
# feature_description = {  
#     # 'feature': tf.io.FixedLenFeature([], tf.string),  
#     outputs = [('text', tf.io.VarLenFeature(tf.int64))]
#     # Add more features as needed  
# }  
outputs = [('text', tf.io.VarLenFeature(tf.int64))]

def _parse_function(example_proto):  
    # Parse the example using the feature description  
    parsed_example = tf.io.parse_single_example(example_proto, dict(outputs))  
    return parsed_example  
 
# Load the dataset from the TFRecord file  
tfrecord_file = one_data_path

raw_dataset = tf.data.TFRecordDataset(tfrecord_file)  
 
# Parse the dataset using the _parse_function  
parsed_dataset = raw_dataset.map(_parse_function)  
 
# Print the first example from the parsed_dataset  
for example in parsed_dataset.take(1):  
    print(tf.sparse.to_dense(example["text"]))  

tf.Tensor([13247 34645  2916 ... 23640 36163 26308], shape=(2048,), dtype=int64)


In [3]:
dataset = tf.data.TFRecordDataset(example_path)

In [4]:
  def _ProcessSingleExample(self, key, record):
    """Parses a serialized tf.Example record."""
    p = self.params
    del key
    features = tf.io.parse_single_example(
        serialized=record,
        features={
            'text':
                tf.io.FixedLenFeature((), dtype=tf.string, default_value=''),
        })
    # ids, labels, paddings = self.StringsToIds(
    #     [features['text']], external_max_length=p.source_max_length)
    # ret = py_utils.NestedMap(ids=ids, labels=labels, paddings=paddings)
    # ret.weights = 1.0 - ret.paddings
    # return ret

In [19]:
dataset = dataset.map(
        _ProcessSingleExample,
        num_parallel_calls=tf.data.AUTOTUNE).unbatch()

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf___ProcessSingleExample() missing 2 required positional arguments: 'key' and 'record'


In [9]:
dataset.element_spec

TensorSpec(shape=(), dtype=tf.string, name=None)

In [12]:
def decode_fn(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,

      # Schema
      {None: tf.io.FixedLenFeature((), dtype=tf.string),
       # "y": tf.io.FixedLenFeature([], dtype=tf.float32)
      }
  )

In [13]:
for batch in tf.data.TFRecordDataset([example_path]).map(decode_fn):
  print("x = {x}".format(**batch))

TypeError: in user code:

    File "/tmp/ipykernel_78498/3925886508.py", line 7, in decode_fn  *
        {None: tf.io.FixedLenFeature((), dtype=tf.string),

    TypeError: Expected string passed to parameter 'dense_keys' of op 'ParseExampleV2', got [None] of type 'list' instead. Error: Expected string, but got None of type 'NoneType'.


In [14]:

def read_tfrecord_of_strings_without_key(tfrecord_file):
  """Reads a TFRecord of strings without a key.

  Args:
    tfrecord_file: The path to the TFRecord file.

  Returns:
    A tf.Tensor containing the decoded strings.
  """

  dataset = tf.data.TFRecordDataset(tfrecord_file)

  # Decode the raw bytes of each example in the dataset.
  decoded_bytes = dataset.map(tf.io.decode_raw)

  # Decode the decoded bytes into a string.
  decoded_strings = decoded_bytes.map(tf.strings.decode_utf8)

  return decoded_strings

In [15]:
decoded_strings = read_tfrecord_of_strings_without_key("gs://yejingxin-us-central2/external/lg/dummy-data/train/LG_GPT_KO2_8_11_100000.tfrecords")


TypeError: Missing required positional argument